# Chapter 30: Dependency Injection in TypeScript

---

## 30.1 Understanding Dependency Injection

Dependency Injection (DI) is a design pattern that implements Inversion of Control (IoC), where a class receives its dependencies from external sources rather than creating them internally. This pattern promotes loose coupling, enhances testability, and improves code maintainability. TypeScript's static type system makes DI particularly powerful by ensuring dependencies are correctly typed at compile time.

### 30.1.1 Core Concepts

Dependency Injection eliminates hard-coded dependencies, allowing components to be configured with their dependencies from the outside.

```typescript
// Without Dependency Injection (tight coupling)
class UserService {
  private database = new Database(); // Hard-coded dependency
  
  async getUser(id: string) {
    return this.database.query(`SELECT * FROM users WHERE id = ${id}`);
  }
}

// Problems:
// 1. Cannot easily swap Database implementation
// 2. Difficult to test (cannot mock Database)
// 3. Violates Single Responsibility Principle

// With Dependency Injection (loose coupling)
interface Database {
  query(sql: string): Promise<any>;
}

class UserService {
  constructor(private database: Database) {}
  
  async getUser(id: string) {
    return this.database.query(`SELECT * FROM users WHERE id = ${id}`);
  }
}

// Benefits:
// 1. Database can be swapped (MySQL, PostgreSQL, Mock)
// 2. Easy to test (inject mock)
// 3. Clear dependency contract via interface
```

**Dependency Injection Patterns:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Dependency Injection Patterns                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. Constructor Injection (Most Common)                            │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ class Service {                                               │ │
│   │   constructor(private dep: Dependency) {}                     │ │
│   │ }                                                             │ │
│   └──────────────────────────────────────────────────────────────┘ │
│   • Dependencies provided via constructor                           │
│   • Guarantees object is fully initialized                          │
│   • Immutable dependencies (cannot be changed)                    │
│                                                                     │
│   2. Property Injection                                              │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ class Service {                                               │ │
│   │   @Inject() dep: Dependency;                                  │ │
│   │ }                                                             │ │
│   └──────────────────────────────────────────────────────────────┘ │
│   • Dependencies set after construction                             │
│   • Optional dependencies possible                                │
│   • Less explicit, harder to track                                  │
│                                                                     │
│   3. Method Injection                                                │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ class Service {                                               │ │
│   │   process(dep: Dependency) { ... }                             │ │
│   │ }                                                             │ │
│   └──────────────────────────────────────────────────────────────┘ │
│   • Dependencies passed to specific methods                         │
│   • Good for contextual dependencies                                │
│                                                                     │
│   4. Interface Injection                                             │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ class Service implements Injectable {                         │ │
│   │   inject(dependency: Dependency) { ... }                     │ │
│   │ }                                                             │ │
│   └──────────────────────────────────────────────────────────────┘ │
│   • Class receives injector via interface                           │
│   • Complex, rarely used in TypeScript                              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 30.1.2 Benefits of Dependency Injection

DI provides concrete advantages for software architecture and development workflows.

```typescript
// 1. Testability - Easy mocking
interface EmailService {
  send(to: string, subject: string, body: string): Promise<void>;
}

class UserRegistrationService {
  constructor(
    private emailService: EmailService,
    private userRepository: UserRepository
  ) {}
  
  async register(email: string, password: string) {
    // Create user...
    await this.userRepository.save({ email, password });
    
    // Send welcome email
    await this.emailService.send(
      email,
      "Welcome!",
      "Thanks for registering"
    );
  }
}

// Unit test with mocks
class MockEmailService implements EmailService {
  sentEmails: Array<{ to: string; subject: string; body: string }> = [];
  
  async send(to: string, subject: string, body: string): Promise<void> {
    this.sentEmails.push({ to, subject, body });
  }
}

// Test
const mockEmail = new MockEmailService();
const mockRepo = new MockUserRepository();
const service = new UserRegistrationService(mockEmail, mockRepo);

await service.register("test@example.com", "password");
expect(mockEmail.sentEmails).toHaveLength(1);

// 2. Flexibility - Swap implementations
// Production
const productionService = new UserRegistrationService(
  new SendGridEmailService(),
  new PostgresUserRepository()
);

// Development
const developmentService = new UserRegistrationService(
  new ConsoleEmailService(), // Just logs to console
  new InMemoryUserRepository()
);

// 3. Configuration externalization
interface DatabaseConfig {
  host: string;
  port: number;
  database: string;
}

class DatabaseConnection {
  constructor(private config: DatabaseConfig) {}
  
  connect() {
    // Use this.config...
  }
}

// Configuration from environment
const db = new DatabaseConnection({
  host: process.env.DB_HOST!,
  port: parseInt(process.env.DB_PORT!),
  database: process.env.DB_NAME!
});

// 4. Lifecycle management
class SingletonService {
  private static instance?: SingletonService;
  
  static getInstance() {
    if (!SingletonService.instance) {
      SingletonService.instance = new SingletonService();
    }
    return SingletonService.instance;
  }
  
  private constructor() {}
}

// DI container manages lifecycle automatically
```

---

## 30.2 DI Container Patterns

A DI Container (or IoC Container) is a framework that manages object creation and dependency resolution automatically, handling the wiring of dependencies throughout your application.

### 30.2.1 Manual Container Implementation

Build a simple but functional DI container to understand the core mechanics.

```typescript
// Service lifetime types
type Lifetime = "transient" | "singleton" | "scoped";

// Registration record
interface Registration<T> {
  token: InjectionToken<T>;
  implementation: new (...args: any[]) => T;
  lifetime: Lifetime;
  instance?: T;
  factory?: () => T;
}

// Injection token (symbol or string)
type InjectionToken<T> = symbol | string | (new (...args: any[]) => T);

class DIContainer {
  private registrations = new Map<InjectionToken<any>, Registration<any>>();
  private singletons = new Map<InjectionToken<any>, any>();
  
  // Register a class
  register<T>(
    token: InjectionToken<T>,
    implementation: new (...args: any[]) => T,
    lifetime: Lifetime = "transient"
  ): this {
    this.registrations.set(token, {
      token,
      implementation,
      lifetime
    });
    return this;
  }
  
  // Register a factory
  registerFactory<T>(
    token: InjectionToken<T>,
    factory: () => T,
    lifetime: Lifetime = "transient"
  ): this {
    this.registrations.set(token, {
      token,
      implementation: class {} as any,
      lifetime,
      factory
    });
    return this;
  }
  
  // Register an instance (for singletons or testing)
  registerInstance<T>(token: InjectionToken<T>, instance: T): this {
    this.singletons.set(token, instance);
    return this;
  }
  
  // Resolve a dependency
  resolve<T>(token: InjectionToken<T>): T {
    // Check for existing singleton
    if (this.singletons.has(token)) {
      return this.singletons.get(token);
    }
    
    const registration = this.registrations.get(token);
    if (!registration) {
      // Auto-registration for classes
      if (typeof token === "function" && token.prototype) {
        return this.createInstance(token as any);
      }
      throw new Error(`No registration found for token: ${String(token)}`);
    }
    
    // Use factory if provided
    if (registration.factory) {
      const instance = registration.factory();
      if (registration.lifetime === "singleton") {
        this.singletons.set(token, instance);
      }
      return instance;
    }
    
    // Create instance with dependencies
    const instance = this.createInstance(registration.implementation);
    
    // Cache singletons
    if (registration.lifetime === "singleton") {
      this.singletons.set(token, instance);
    }
    
    return instance;
  }
  
  private createInstance<T>(constructor: new (...args: any[]) => T): T {
    // Get constructor parameter types from metadata
    const paramTypes: InjectionToken<any>[] = Reflect.getMetadata("design:paramtypes", constructor) || [];
    
    // Resolve dependencies
    const dependencies = paramTypes.map(type => this.resolve(type));
    
    // Create instance
    return new constructor(...dependencies);
  }
  
  // Create a scope for scoped lifetime
  createScope(): DIContainer {
    const scope = new DIContainer();
    scope.registrations = this.registrations;
    scope.singletons = this.singletons;
    return scope;
  }
  
  // Build provider function
  buildProvider<T>(token: InjectionToken<T>): () => T {
    return () => this.resolve(token);
  }
}

// Usage
interface Database {
  query(sql: string): Promise<any>;
}

class PostgresDatabase implements Database {
  async query(sql: string): Promise<any> {
    console.log("Executing:", sql);
    return [];
  }
}

class UserRepository {
  constructor(private db: Database) {}
  
  async findById(id: string) {
    return this.db.query(`SELECT * FROM users WHERE id = ${id}`);
  }
}

class UserService {
  constructor(private repo: UserRepository) {}
  
  async getUser(id: string) {
    return this.repo.findById(id);
  }
}

// Setup container
const container = new DIContainer();

container
  .register<Database>("Database", PostgresDatabase, "singleton")
  .register<UserRepository>(UserRepository, UserRepository)
  .register<UserService>(UserService, UserService);

// Resolve
const userService = container.resolve(UserService);
// Container automatically injects UserRepository and Database
```

### 30.2.2 Decorator-Based Registration

Use TypeScript decorators for declarative dependency registration.

```typescript
// Enable metadata reflection
import "reflect-metadata";

// Decorators for DI
function Injectable<T extends { new (...args: any[]): {} }>(constructor: T) {
  // Mark class as injectable
  Reflect.defineMetadata("injectable", true, constructor);
  return constructor;
}

function Inject(token: InjectionToken<any>) {
  return function(target: any, propertyKey: string | symbol | undefined, parameterIndex: number) {
    const existingTokens = Reflect.getMetadata("design:inject", target) || [];
    existingTokens[parameterIndex] = token;
    Reflect.defineMetadata("design:inject", existingTokens, target);
  };
}

function Singleton<T extends { new (...args: any[]): {} }>(constructor: T) {
  Reflect.defineMetadata("lifetime", "singleton", constructor);
  return constructor;
}

// Enhanced container with decorator support
class DecoratorContainer extends DIContainer {
  registerClass<T extends { new (...args: any[]): {} }>(constructor: T): this {
    const lifetime = Reflect.getMetadata("lifetime", constructor) || "transient";
    this.register(constructor, constructor, lifetime);
    return this;
  }
  
  resolve<T>(token: InjectionToken<T>): T {
    // Check for @Inject decorators
    if (typeof token === "function") {
      const customTokens: InjectionToken<any>[] = Reflect.getMetadata("design:inject", token) || [];
      const paramTypes: InjectionToken<any>[] = Reflect.getMetadata("design:paramtypes", token) || [];
      
      // Merge custom tokens with auto-detected types
      const dependencies = paramTypes.map((type, index) => {
        const customToken = customTokens[index];
        if (customToken) {
          return this.resolve(customToken);
        }
        return this.resolve(type);
      });
      
      return new (token as any)(...dependencies);
    }
    
    return super.resolve(token);
  }
}

// Usage with decorators
@Injectable
class Logger {
  log(message: string) {
    console.log(`[LOG] ${message}`);
  }
}

@Injectable
@Singleton
class DatabaseService {
  constructor(@Inject("Config") private config: any) {}
  
  connect() {
    console.log("Connecting to database...");
  }
}

@Injectable
class UserController {
  constructor(
    private logger: Logger,
    private database: DatabaseService
  ) {}
  
  handleRequest() {
    this.logger.log("Handling request");
    this.database.connect();
  }
}

// Setup
const container = new DecoratorContainer();
container.registerInstance("Config", { dbUrl: "postgres://localhost" });
container.registerClass(Logger);
container.registerClass(DatabaseService);
container.registerClass(UserController);

const controller = container.resolve(UserController);
controller.handleRequest();
```

---

## 30.3 Type-Safe Injection Tokens

Injection tokens are the keys used to identify and resolve dependencies. Type-safe tokens ensure that the container returns the correct type and that TypeScript can verify usage at compile time.

### 30.3.1 Symbol-Based Tokens

Use ES6 Symbols to create unique, collision-free injection tokens.

```typescript
// Create unique tokens
const DatabaseToken = Symbol("Database") as InjectionToken<Database>;
const LoggerToken = Symbol("Logger") as InjectionToken<Logger>;
const ConfigToken = Symbol("Config") as InjectionToken<AppConfig>;

// Type-safe registration
container.register(DatabaseToken, PostgresDatabase, "singleton");
container.register(LoggerToken, ConsoleLogger);
container.registerInstance(ConfigToken, {
  apiUrl: "https://api.example.com",
  timeout: 5000
});

// Type-safe resolution
const db = container.resolve(DatabaseToken); // Type: Database
const logger = container.resolve(LoggerToken); // Type: Logger
const config = container.resolve(ConfigToken); // Type: AppConfig

// Class-based tokens (use class itself as token)
class EmailService {
  send(email: string) { /* ... */ }
}

// Register and resolve by class
container.register(EmailService, SendGridService);
const email = container.resolve(EmailService); // Type: EmailService
```

### 30.3.2 Token Factory Pattern

Create typed token factories for better organization and discoverability.

```typescript
// Token factory with type safety
function createToken<T>(name: string): InjectionToken<T> {
  return Symbol(name) as InjectionToken<T>;
}

// Organized tokens
const Tokens = {
  Database: createToken<Database>("Database"),
  Logger: createToken<Logger>("Logger"),
  Cache: createToken<Cache>("Cache"),
  Config: createToken<AppConfig>("Config"),
  
  // Services
  UserService: createToken<UserService>("UserService"),
  AuthService: createToken<AuthService>("AuthService"),
  PaymentService: createToken<PaymentService>("PaymentService"),
  
  // Repositories
  UserRepository: createToken<UserRepository>("UserRepository"),
  OrderRepository: createToken<OrderRepository>("OrderRepository")
} as const;

// Type-safe usage
class ServiceModule {
  static configure(container: DIContainer): void {
    container
      .register(Tokens.Database, PostgresDatabase, "singleton")
      .register(Tokens.Logger, WinstonLogger, "singleton")
      .register(Tokens.UserRepository, UserRepository)
      .register(Tokens.UserService, UserService);
  }
}

// Constructor injection with tokens
class OrderService {
  constructor(
    @Inject(Tokens.Database) private db: Database,
    @Inject(Tokens.Logger) private logger: Logger,
    @Inject(Tokens.UserRepository) private users: UserRepository
  ) {}
}
```

### 30.3.3 Async Dependency Resolution

Handle asynchronous initialization and dynamic imports.

```typescript
class AsyncContainer extends DIContainer {
  private asyncRegistrations = new Map<InjectionToken<any>, () => Promise<any>>();
  
  registerAsync<T>(
    token: InjectionToken<T>,
    factory: () => Promise<T>,
    lifetime: Lifetime = "transient"
  ): this {
    this.asyncRegistrations.set(token, factory);
    
    // Also register sync wrapper
    this.registerFactory(token, () => {
      throw new Error(`Use resolveAsync for token ${String(token)}`);
    }, lifetime);
    
    return this;
  }
  
  async resolveAsync<T>(token: InjectionToken<T>): Promise<T> {
    // Check if async factory exists
    const asyncFactory = this.asyncRegistrations.get(token);
    if (asyncFactory) {
      const instance = await asyncFactory();
      return instance;
    }
    
    // Fall back to sync resolution
    return this.resolve(token);
  }
  
  // Initialize all singletons asynchronously
  async initializeSingletons(): Promise<void> {
    for (const [token, registration] of this.registrations) {
      if (registration.lifetime === "singleton") {
        await this.resolveAsync(token);
      }
    }
  }
}

// Usage with dynamic imports
const container = new AsyncContainer();

// Register async dependencies
container.registerAsync("HeavyModule", async () => {
  const module = await import("./heavy-module");
  return new module.HeavyService();
}, "singleton");

// Initialize
async function bootstrap() {
  const heavyService = await container.resolveAsync("HeavyModule");
  // Use service...
}
```

---

## 30.4 Framework Integration

Production applications typically use established DI frameworks rather than custom implementations. TypeScript has excellent support for popular DI containers.

### 30.4.1 InversifyJS

InversifyJS is a powerful, feature-rich DI container for TypeScript with decorators and advanced features.

```typescript
// Setup
import { Container, injectable, inject, named, optional } from "inversify";
import "reflect-metadata";

// Define symbols
const TYPES = {
  Warrior: Symbol.for("Warrior"),
  Weapon: Symbol.for("Weapon"),
  ThrowableWeapon: Symbol.for("ThrowableWeapon")
};

// Interfaces
interface Warrior {
  fight(): string;
  sneak(): string;
}

interface Weapon {
  hit(): string;
}

interface ThrowableWeapon {
  throw(): string;
}

// Implementations
@injectable()
class Katana implements Weapon {
  hit() {
    return "cut!";
  }
}

@injectable()
class Shuriken implements ThrowableWeapon {
  throw() {
    return "hit!";
  }
}

@injectable()
class Ninja implements Warrior {
  private _katana: Weapon;
  private _shuriken: ThrowableWeapon;
  
  constructor(
    @inject(TYPES.Weapon) katana: Weapon,
    @inject(TYPES.ThrowableWeapon) shuriken: ThrowableWeapon
  ) {
    this._katana = katana;
    this._shuriken = shuriken;
  }
  
  fight() {
    return this._katana.hit();
  }
  
  sneak() {
    return this._shuriken.throw();
  }
}

// Configure container
const container = new Container();
container.bind<Weapon>(TYPES.Weapon).to(Katana);
container.bind<ThrowableWeapon>(TYPES.ThrowableWeapon).to(Shuriken);
container.bind<Warrior>(TYPES.Warrior).to(Ninja);

// Resolve
const ninja = container.get<Warrior>(TYPES.Warrior);
console.log(ninja.fight()); // cut!
console.log(ninja.sneak()); // hit!

// Advanced features
container.bind<Weapon>(TYPES.Weapon)
  .to(Katana)
  .whenTargetNamed("strong"); // Conditional binding

container.bind<Weapon>(TYPES.Weapon)
  .to(Shuriken)
  .whenTargetNamed("stealth");

@injectable()
class Samurai {
  constructor(
    @inject(TYPES.Weapon) @named("strong") private weapon: Weapon
  ) {}
}
```

### 30.4.2 TSyringe

TSyringe is a lightweight DI container from Microsoft with a focus on simplicity and decorator-based configuration.

```typescript
// Setup
import "reflect-metadata";
import { container, injectable, inject, singleton, registry } from "tsyringe";

// Interfaces
interface Database {
  query(sql: string): Promise<any>;
}

interface Logger {
  log(message: string): void;
}

// Implementations
@injectable()
@singleton()
class PostgresDB implements Database {
  async query(sql: string) {
    console.log("Postgres:", sql);
    return [];
  }
}

@injectable()
class ConsoleLogger implements Logger {
  log(message: string) {
    console.log(message);
  }
}

// Service with dependencies
@injectable()
class UserService {
  constructor(
    @inject("Database") private db: Database,
    private logger: Logger // Auto-inject by type
  ) {}
  
  async getUsers() {
    this.logger.log("Fetching users");
    return this.db.query("SELECT * FROM users");
  }
}

// Manual registration
container.register<Database>("Database", { useClass: PostgresDB });
container.register("Config", { useValue: { apiUrl: "..." } });

// Token aliases
container.register("DB", { useToken: "Database" });

// Resolution
const userService = container.resolve(UserService);
userService.getUsers();

// Factory registration
container.register("Factory", {
  useFactory: (c) => {
    const db = c.resolve<Database>("Database");
    return new UserService(db, c.resolve(ConsoleLogger));
  }
});

// Cleanup
container.clearInstances();
container.reset();
```

### 30.4.3 NestJS Integration

NestJS is a full-featured framework with built-in DI container, commonly used for server-side applications.

```typescript
// NestJS uses decorators extensively
import { Module, Controller, Injectable, Inject, Get, Post } from "@nestjs/common";

// Service layer
@Injectable()
export class CatsService {
  private readonly cats: Cat[] = [];
  
  create(cat: Cat) {
    this.cats.push(cat);
  }
  
  findAll(): Cat[] {
    return this.cats;
  }
}

// Controller
@Controller("cats")
export class CatsController {
  constructor(private catsService: CatsService) {}
  
  @Post()
  create(@Body() createCatDto: CreateCatDto) {
    this.catsService.create(createCatDto);
  }
  
  @Get()
  findAll(): Cat[] {
    return this.catsService.findAll();
  }
}

// Module organization
@Module({
  controllers: [CatsController],
  providers: [CatsService],
  exports: [CatsService] // Available to other modules
})
export class CatsModule {}

// Root module
@Module({
  imports: [CatsModule, DatabaseModule],
  providers: [
    {
      provide: "DATABASE_URL",
      useValue: process.env.DATABASE_URL
    },
    {
      provide: "Logger",
      useClass: ConsoleLogger
    },
    {
      provide: "Factory",
      useFactory: (config: ConfigService) => {
        return new DatabaseService(config.get("DB_URL"));
      },
      inject: [ConfigService]
    }
  ]
})
export class AppModule {}
```

---

## 30.5 Manual DI vs Frameworks

Choosing between manual dependency injection and frameworks depends on project size, complexity, and team preferences.

### 30.5.1 When to Use Manual DI

Simple applications often benefit from manual DI without framework overhead.

```typescript
// Composition Root pattern - manual wiring at application startup
class CompositionRoot {
  static createApplication(): Application {
    // Infrastructure layer
    const config = loadConfig();
    const logger = new ConsoleLogger();
    const db = new PostgresDatabase(config.databaseUrl);
    
    // Repository layer
    const userRepo = new UserRepository(db);
    const orderRepo = new OrderRepository(db);
    
    // Service layer
    const emailService = new SendGridEmailService(config.sendGridKey);
    const userService = new UserService(userRepo, emailService, logger);
    const orderService = new OrderService(orderRepo, userService);
    
    // Controller layer
    const userController = new UserController(userService);
    const orderController = new OrderController(orderService);
    
    // Application
    return new Application([
      userController,
      orderController
    ]);
  }
}

// Pure DI (no container, just constructor calls)
// Pros:
// - Zero dependencies
// - Compile-time safety
// - Easy to understand
// - No reflection/magic
// Cons:
// - Verbose setup
// - Manual lifecycle management
// - Harder to manage in large apps

// Best for:
// - Small to medium applications
// - Libraries
// - When you want explicit control
// - Performance-critical code (no reflection overhead)
```

### 30.5.2 When to Use Frameworks

Large applications benefit from DI framework features like auto-wiring and lifecycle management.

```typescript
// Framework benefits:
// 1. Auto-registration and discovery
// 2. Lifecycle management (singleton, transient, scoped)
// 3. Interception/AOP (caching, logging, transactions)
// 4. Module organization
// 5. Testing utilities

// Example with interceptors (NestJS)
@Injectable()
export class LoggingInterceptor implements NestInterceptor {
  intercept(context: ExecutionContext, next: CallHandler): Observable<any> {
    const now = Date.now();
    return next
      .handle()
      .pipe(
        tap(() => console.log(`Execution time: ${Date.now() - now}ms`))
      );
  }
}

// Example with caching (Inversify)
container.bind<Weapon>(TYPES.Weapon)
  .to(Katana)
  .inSingletonScope();

// Best for:
// - Large enterprise applications
// - Microservices
// - Complex object graphs
// - When team is familiar with framework
// - When you need advanced features (AOP, modules)
```

### 30.5.3 Hybrid Approaches

Combine manual and framework-based DI for optimal flexibility.

```typescript
// Manual DI for core services
const coreServices = {
  config: loadConfig(),
  logger: new WinstonLogger()
};

// Framework container for business logic
const container = new Container();

// Register manual services in container
container.registerInstance("Config", coreServices.config);
container.registerInstance("Logger", coreServices.logger);

// Auto-wire the rest
container.scan(["./services/**/*.ts"]);

// Testing with manual DI
describe("UserService", () => {
  it("should create user", async () => {
    // Manual injection of mocks
    const mockRepo = new MockUserRepository();
    const mockEmail = new MockEmailService();
    const service = new UserService(mockRepo, mockEmail);
    
    const result = await service.create({ email: "test@example.com" });
    
    expect(result).toBeDefined();
    expect(mockRepo.savedUsers).toHaveLength(1);
  });
});
```

---

## 30.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored Dependency Injection in TypeScript:

**Key Takeaways:**

1. **Core Concepts**:
   - Constructor injection is the preferred pattern
   - DI enables loose coupling and testability
   - Dependencies should depend on abstractions (interfaces)

2. **DI Containers**:
   - Manage object creation and lifecycle
   - Support transient, singleton, and scoped lifetimes
   - Can be implemented manually or via frameworks

3. **Type Safety**:
   - Use symbols or classes as injection tokens
   - Leverage TypeScript decorators for metadata
   - Ensure compile-time type checking of dependencies

4. **Frameworks**:
   - InversifyJS: Feature-rich, decorator-based
   - TSyringe: Lightweight, Microsoft-backed
   - NestJS: Full framework with built-in DI

5. **Best Practices**:
   - Prefer constructor injection
   - Use interfaces for dependencies
   - Manage lifetimes appropriately
   - Consider manual DI for simple cases

### Practical Exercises

**Exercise 1: Manual DI**

Implement manual dependency injection:

```typescript
// Create a simple application with:
// 1. Database interface and Postgres implementation
// 2. Repository layer depending on Database
// 3. Service layer depending on Repository and Logger
// 4. Controller layer depending on Service
// 5. Composition root that wires everything together
// 6. Tests using mock implementations

// Ensure:
// - No class creates its own dependencies
// - All dependencies are interfaces
// - Easy to swap implementations
```

**Exercise 2: Custom Container**

Build a DI container:

```typescript
// Create a DIContainer class with:
// 1. register() for class registration
// 2. registerInstance() for pre-created objects
// 3. registerFactory() for custom creation logic
// 4. resolve() with automatic dependency injection
// 5. Support for singleton and transient lifetimes
// 6. Circular dependency detection

// Test with a 3-level dependency chain:
// Service -> Repository -> Database
```

**Exercise 3: Decorator Integration**

Add decorator support:

```typescript
// Implement @Injectable, @Inject, and @Singleton decorators
// that work with your container from Exercise 2

// Usage:
@Injectable
class UserService {
  constructor(
    @Inject("Database") private db: Database,
    @Inject("Logger") private logger: Logger
  ) {}
}

@Singleton
@Injectable
class DatabaseService {}
```

**Exercise 4: Framework Comparison**

Compare DI frameworks:

```typescript
// Implement the same service layer using:
// 1. InversifyJS
// 2. TSyringe
// 3. Manual DI

// Compare:
// - Setup complexity
// - Bundle size
// - Performance
// - Developer experience
// - Testing ease
```

### Additional Resources

- **InversifyJS Documentation**: https://inversify.io/
- **TSyringe GitHub**: https://github.com/microsoft/tsyringe
- **NestJS Fundamentals**: https://docs.nestjs.com/providers
- **DI Pattern (Martin Fowler)**: https://martinfowler.com/articles/injection.html
- **TypeScript DI Guide**: https://khalilstemmler.com/articles/tutorials/dependency-injection-in-typescript/

---

## Coming Up Next: Chapter 31 - Advanced Design Patterns

In the next chapter, we will explore **Advanced Design Patterns** implemented in TypeScript:

- **31.1 Singleton Pattern** - Ensuring single instances with type safety
- **31.2 Factory Pattern** - Creating objects with flexible instantiation
- **31.3 Builder Pattern** - Constructing complex objects step by step
- **31.4 Observer Pattern** - Event subscription and notification
- **31.5 Strategy Pattern** - Interchangeable algorithms
- **31.6 Command Pattern** - Encapsulating requests as objects
- **31.7 Decorator Pattern** - Adding responsibilities dynamically
- **31.8 Proxy Pattern** - Controlling access to objects

Design patterns provide proven solutions to recurring problems. TypeScript's type system enhances these patterns with compile-time verification, ensuring implementations are correct and maintainable.